In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Credit_Scoring_Project/data.csv')

In [4]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
# Group by customer and calculate aggregate features
aggregate_features = df.groupby('CustomerId').agg(
    TotalTransactionAmount=('Amount', 'sum'),
    AvgTransactionAmount=('Amount', 'mean'),
    TransactionCount=('TransactionId', 'count'),
    TransactionStdDev=('Amount', 'std')
).reset_index()

# Fill missing values in standard deviation (e.g., customers with one transaction)
aggregate_features['TransactionStdDev'].fillna(0, inplace=True)

aggregate_features.head()


<ipython-input-6-e2e0510b7c43>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aggregate_features['TransactionStdDev'].fillna(0, inplace=True)


,CustomerId,TotalTransactionAmount,AvgTransactionAmount,TransactionCount,TransactionStdDev
0,CustomerId_1,-10000.0,-10000.000000,1,0.000000
1,CustomerId_10,-10000.0,-10000.000000,1,0.000000
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,4225.0,384.090909,11,560.498966
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146


In [7]:
# Ensure 'TransactionStartTime' is in datetime format
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extract temporal features
df['TransactionHour'] = df['TransactionStartTime'].dt.hour
df['TransactionDay'] = df['TransactionStartTime'].dt.day
df['TransactionMonth'] = df['TransactionStartTime'].dt.month
df['TransactionYear'] = df['TransactionStartTime'].dt.year


In [16]:
# Calculate transactions per day for each customer
transactions_per_day = df.groupby(['CustomerId', 'TransactionDay'])['TransactionId'].count().reset_index()
transactions_per_day = transactions_per_day.rename(columns={'TransactionId': 'TransactionsPerDay'})

# Merge this feature back into the main dataframe or aggregate_features
df = pd.merge(df, transactions_per_day, on=['CustomerId', 'TransactionDay'], how='left')

In [19]:
# Assuming 'TransactionStartTime' is sorted
df['TimeSinceLastTransaction'] = df.groupby('CustomerId')['TransactionStartTime'].diff().dt.total_seconds()

# Handle the first transaction for each customer (could be NaN)
df['TimeSinceLastTransaction'].fillna(0, inplace=True)

<ipython-input-19-9c25a01eb4d2>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TimeSinceLastTransaction'].fillna(0, inplace=True)


In [20]:
aggregate_features['TotalAmount_x_TransactionCount'] = aggregate_features['TotalTransactionAmount'] * aggregate_features['TransactionCount']

In [9]:
# One-Hot Encoding example
df = pd.get_dummies(df, columns=['CountryCode', 'ProductCategory'], drop_first=True)

# Label Encoding example (for a single column)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['ChannelId'] = le.fit_transform(df['ChannelId'])


In [26]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,ProviderId,ProductId,ChannelId,Amount,...,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,TransactionsPerDay,TimeSinceLastTransaction
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,ProviderId_6,ProductId_10,2,0.092004,...,False,False,False,False,False,False,False,False,3,0.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,ProviderId_4,ProductId_6,1,0.091910,...,False,True,False,False,False,False,False,False,3,19.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,ProviderId_6,ProductId_1,2,0.091958,...,False,False,False,False,False,False,False,False,1,0.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,ProviderId_1,ProductId_21,2,0.093750,...,False,False,False,False,False,False,False,True,4,0.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,ProviderId_4,ProductId_6,1,0.091853,...,False,True,False,False,False,False,False,False,4,86.0


In [25]:
# Replace missing numerical values with the mean
df['Amount'] = df['Amount'].fillna(df['Amount'].mean())

In [12]:
df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'ProviderId', 'ProductId', 'ChannelId', 'Amount',
       'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'TransactionHour', 'TransactionDay', 'TransactionMonth',
       'TransactionYear', 'ProductCategory_data_bundles',
       'ProductCategory_financial_services', 'ProductCategory_movies',
       'ProductCategory_other', 'ProductCategory_ticket',
       'ProductCategory_transport', 'ProductCategory_tv',
       'ProductCategory_utility_bill'],
      dtype='object')

In [13]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Select numerical features for scaling
numerical_columns = ['Amount', 'Value']

# Apply Min-Max Scaling
scaler = MinMaxScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [18]:
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,ProviderId,ProductId,ChannelId,Amount,...,TransactionYear,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,TransactionsPerDay
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,ProviderId_6,ProductId_10,2,0.092004,...,2018,False,False,False,False,False,False,False,False,3
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,ProviderId_4,ProductId_6,1,0.091910,...,2018,False,True,False,False,False,False,False,False,3
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,ProviderId_6,ProductId_1,2,0.091958,...,2018,False,False,False,False,False,False,False,False,1
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,ProviderId_1,ProductId_21,2,0.093750,...,2018,False,False,False,False,False,False,False,True,4
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,ProviderId_4,ProductId_6,1,0.091853,...,2018,False,True,False,False,False,False,False,False,4


In [21]:
aggregate_features

,CustomerId,TotalTransactionAmount,AvgTransactionAmount,TransactionCount,TransactionStdDev,TotalAmount_x_TransactionCount
0,CustomerId_1,-10000.0,-10000.000000,1,0.000000,-10000.0
1,CustomerId_10,-10000.0,-10000.000000,1,0.000000,-10000.0
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333,100000.0
3,CustomerId_1002,4225.0,384.090909,11,560.498966,46475.0
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146,120000.0
...,...,...,...,...,...,...
3737,CustomerId_992,20000.0,3333.333333,6,6088.240030,120000.0
3738,CustomerId_993,20000.0,4000.000000,5,6745.368782,100000.0
3739,CustomerId_994,543873.0,5384.881188,101,14800.656784,54931173.0
3740,CustomerId_996,139000.0,8176.470588,17,4433.329648,2363000.0


In [28]:
print(df.head())


         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode    ProviderId     ProductId  ChannelId  \
0  CustomerId_4406          UGX  ProviderId_6  ProductId_10          2   
1  CustomerId_4406          UGX  ProviderId_4   ProductId_6          1   
2  CustomerId_4683          UGX  ProviderId_6   ProductId_1          2   
3   CustomerId_988          UGX  ProviderId_1  ProductId_21          2   
4   CustomerId_988          UGX  ProviderId_4   ProductId_6          1   

     Amount  ...  ProductCategory_data_bundles  \
0  0.092004  ...              

In [29]:
print(aggregate_features.head())

        CustomerId  TotalTransactionAmount  AvgTransactionAmount  \
0     CustomerId_1                -10000.0         -10000.000000   
1    CustomerId_10                -10000.0         -10000.000000   
2  CustomerId_1001                 20000.0           4000.000000   
3  CustomerId_1002                  4225.0            384.090909   
4  CustomerId_1003                 20000.0           3333.333333   

   TransactionCount  TransactionStdDev  TotalAmount_x_TransactionCount  
0                 1           0.000000                        -10000.0  
1                 1           0.000000                        -10000.0  
2                 5        6558.963333                        100000.0  
3                11         560.498966                         46475.0  
4                 6        6030.478146                        120000.0  


In [31]:
# Save the DataFrame to a CSV file on your Drive
df.to_csv('/content/drive/MyDrive/Credit_Scoring_Project/processed_data.csv', index=False)

In [32]:
aggregate_features.to_csv('/content/drive/MyDrive/Credit_Scoring_Project/aggregate_features.csv', index=False)